In [1]:
# mount to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# reload
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [3]:
import os
os.chdir('/content/drive/MyDrive/LLA')

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore, rankdata
import torch
import torch.nn as nn
import torch.optim as optim
from data_process import *
from training import *
from model import *
import xgboost as xgb
from utilis import *
from sklearn.model_selection import train_test_split

In [5]:
set_seed(42)

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


# **Data Preparation**

In [7]:
MIMICtable = pd.read_csv('dataset/MIMICtable.csv')

In [8]:
num_epoch = 50  # 训练循环次数
gamma = 0.99
beat1 = 0
beat2 = 0.6
beta3 = 0.3
ncv = 5  # nr of crossvalidation runs (each is 80% training / 20% test)交叉验证运行的Nr(每次为80%训练/ 20%测试)
nra = 5
lr = 1e-5
reward_value = 24
beta = [beat1, beat2, beta3]
icustayidlist = MIMICtable['icustayid']
icuuniqueids = np.unique(icustayidlist)  # list of unique icustayids from MIMIC唯一的icustayid列表
reformat5 = MIMICtable.values.copy()
print('####  生成状态  ####')

# -----------------------筛选后的特征=37个--------------------------------
colnorm = ['SOFA', 'age', 'Weight_kg', 'GCS', 'HR', 'SysBP', 'MeanBP', 'DiaBP', 'RR', 'Temp_C',
           'Sodium', 'Chloride', 'Glucose', 'Calcium', 'Hb', 'WBC_count', 'Platelets_count',
           'PTT', 'PT', 'Arterial_pH', 'paO2', 'paCO2', 'HCO3', 'Arterial_lactate', 'Shock_Index',
           'PaO2_FiO2', 'cumulated_balance', 'CO2_mEqL', 'Ionised_Ca']
##8个指标
collog = ['SpO2', 'BUN', 'Creatinine', 'SGOT', 'Total_bili', 'INR', 'input_total', 'output_total']

colnorm = np.where(np.isin(MIMICtable.columns, colnorm))[0]
collog = np.where(np.isin(MIMICtable.columns, collog))[0]

####  生成状态  ####


In [9]:
train = np.load('dataset/train.npy')
validat = np.load('dataset/validation.npy')
test = np.load('dataset/test.npy')

In [10]:
scaleMIMIC = np.concatenate([zscore(reformat5[:, colnorm], ddof=1),
                              zscore(np.log(0.1 + reformat5[:, collog]), ddof=1)], axis=1)

Xvalidat = scaleMIMIC[validat, :]
blocsvalidat = reformat5[validat, 0]
ptidvalidat = reformat5[validat, 1]

Xtrain = scaleMIMIC[train, :]
Xtest = scaleMIMIC[test, :]
blocstrain = reformat5[train, 0]  # 序列号
bloctest = reformat5[test, 0]
ptidtrain = reformat5[train, 1]  # 患者编号
ptidtest = reformat5[test, 1]

# *************************
RNNstate = Xtrain  # ***

print('####  生成动作  ####')
nact = nra * nra  # 5*5=25
iol = MIMICtable.columns.get_loc('input_4hourly')  # 输入的列
vcl = MIMICtable.columns.get_loc('max_dose_vaso')  # 最大使用加压药量的列

a = reformat5[:, iol].copy()  # IV fluid  静脉输液复苏
a = rankdata(a[a > 0]) / a[a > 0].shape[0]  # excludes zero fluid (will be action 1)不包括零液体 将是动作1
iof = np.floor((a + 0.2499999999) * 4)  # converts iv volume in 4 actions 在4个动作中转换静脉输液量
a = reformat5[:, iol].copy()
a = np.where(a > 0)[0]  # location of non-zero fluid in big matrix
io = np.ones((reformat5.shape[0], 1))  # array of ones, by default
io[a] = (iof + 1).reshape(-1, 1)  # where more than zero fluid given: save actual action
io = io.ravel()  # 两者的本质都是想把多维的数组降为1维  注射有5个动作，已经通过秩进行判断

vc = reformat5[:, vcl].copy()
vcr = rankdata(vc[vc != 0]) / vc[vc != 0].size
vcr = np.floor((vcr + 0.249999999999) * 4)  # converts to 4 bins
vcr[vcr == 0] = 1
vc[vc != 0] = vcr + 1
vc[vc == 0] = 1

ma1 = np.array(
    [np.median(reformat5[io == 1, iol]), np.median(reformat5[io == 2, iol]), np.median(reformat5[io == 3, iol]),
        np.median(reformat5[io == 4, iol]), np.median(reformat5[io == 5, iol])])  # median dose of drug in all bins
ma2 = np.array(
    [np.median(reformat5[vc == 1, vcl]), np.median(reformat5[vc == 2, vcl]), np.median(reformat5[vc == 3, vcl]),
        np.median(reformat5[vc == 4, vcl]), np.median(reformat5[vc == 5, vcl])])

med = np.concatenate([io.reshape(-1, 1), vc.reshape(-1, 1)], axis=1)
uniqueValues, actionbloc = np.unique(med, axis=0, return_inverse=True)

actionbloctrain = actionbloc[train]  # ***
actionblocvalidat = actionbloc[validat]  # ***
actionbloctest = actionbloc[test]

ma2Values = ma2[uniqueValues[:, 1].astype('int64') - 1].reshape(-1, 1)
ma1Values = ma1[uniqueValues[:, 0].astype('int64') - 1].reshape(-1, 1)
uniqueValuesdose = np.concatenate([ma2Values, ma1Values], axis=1)  # median dose of each bin for all 25 actions
print('####  生成动作完成  ####')

####  生成动作  ####
####  生成动作完成  ####


In [11]:
outcome = 9
Y90 = reformat5[train, outcome]
r = np.array([reward_value, -reward_value]).reshape(1, -1)
r2 = r * (2 * (1 - Y90.reshape(-1, 1)) - 1)

# -----奖励函数准备-----------------------------
SOFA = reformat5[train, 57]  # ***
R3 = r2[:, 0]
R4 = (R3 + reward_value) / (2 * reward_value)
c = 0
bloc_max = max(blocstrain)

In [12]:
print(RNNstate.shape[0])
print('#### 生成轨迹 ####')

statesize = RNNstate.shape[1]
states = np.zeros((int(RNNstate.shape[0] * 1.2), statesize))
actions = np.zeros((int(RNNstate.shape[0] * 1.2), 1), dtype=int)
next_actions = np.zeros((int(RNNstate.shape[0] * 1.2), 1), dtype=int)
rewards = np.zeros((int(RNNstate.shape[0] * 1.2), 1))
next_states = np.zeros((int(RNNstate.shape[0] * 1.2), statesize))
done_flags = np.zeros((int(RNNstate.shape[0] * 1.2), 1))
bloc_num = np.zeros((int(RNNstate.shape[0] * 1.2), 1))
blocnum1 = 1

c = 0
bloc_num_reward = 0

for i in range(RNNstate.shape[0] - 1):
    states[c] = RNNstate[i, :]
    actions[c] = actionbloctrain[i]
    bloc_num[c] = blocnum1

    if blocstrain[i] != blocstrain[i + 1]:
        next_states1 = np.zeros(statesize)
        next_actions1 = -1
        done_flags1 = 1
        reward1 = -beat1 * SOFA[i] + R3[i]
        blocnum1 += 1
        bloc_num_reward = 0
    else:
        next_states1 = RNNstate[i + 1, :]
        next_actions1 = actionbloctrain[i + 1]
        done_flags1 = 0
        reward1 = -beat2 * (SOFA[i + 1] - SOFA[i])
        bloc_num_reward += 1

    next_states[c] = next_states1
    next_actions[c] = next_actions1
    rewards[c] = reward1
    done_flags[c] = done_flags1
    c += 1

# 处理最后一行
states[c] = RNNstate[-1, :]
actions[c] = actionbloctrain[-1]
bloc_num[c] = blocnum1
next_states1 = np.zeros(statesize)
next_actions1 = -1
done_flags1 = 1
reward1 = -beat1 * SOFA[-1] + R3[-1]
next_states[c] = next_states1
next_actions[c] = next_actions1
rewards[c] = reward1
done_flags[c] = done_flags1
c += 1

# 切片到实际大小
states = states[:c, :]
next_states = next_states[:c, :]
actions = actions[:c, :]
next_actions = next_actions[:c, :]
rewards = rewards[:c, :]
done_flags = done_flags[:c, :]
bloc_num = bloc_num[:c, :]

# 去掉不必要的维度
bloc_num = np.squeeze(bloc_num)
actions = np.squeeze(actions)
rewards = np.squeeze(rewards)
done_flags = np.squeeze(done_flags)

# 转换为torch张量
state = torch.FloatTensor(states).to(device)
next_state = torch.FloatTensor(next_states).to(device)
action = torch.LongTensor(actions).to(device)
next_action = torch.LongTensor(next_actions).to(device)
reward = torch.FloatTensor(rewards).to(device)
done = torch.FloatTensor(done_flags).to(device)
SOFAS = torch.LongTensor(SOFA).to(device)
batchs = (state, next_state, action, next_action, reward, done, bloc_num, SOFAS)

199312
#### 生成轨迹 ####


# **Separate action**

In [13]:
inv_action_map = {}
count = 0
for i in range(5):
    for j in range(5):
        inv_action_map[count] = [i, j]
        count += 1

In [14]:
actions_tuple = [None for i in range(len(actions))]
for i in range(len(actions)):
    actions_tuple[i] = inv_action_map[actions[i]]
actions_tuple = np.array(actions_tuple)

In [15]:
actions_iv = actions_tuple[:, 0]
actions_vaso = actions_tuple[:, 1]

In [16]:
# check the zero ratio of the actions
print(np.sum(actions_iv == 0) / len(actions_iv))
print(np.sum(actions_vaso == 0) / len(actions_vaso))

0.28191478686682186
0.8624769206068877


In [17]:
len(actions_iv)

199312

In [18]:
# append action_iv and action_vaso as another two feature
states_with_action = np.concatenate([states, actions_iv.reshape(-1, 1), actions_vaso.reshape(-1, 1)], axis=1)

In [19]:
states_with_action.shape

(199312, 39)

In [ ]:
blocstrain[:20]

array([ 1.,  2.,  3.,  4.,  5.,  6.,  9.,  1.,  2.,  3.,  4.,  5.,  6.,
        7.,  8.,  9., 10., 11., 12.,  1.])

# **XGBoost**

In [22]:
from importlib import reload
reload(data_process)
import data_process
from data_process import *

In [36]:
# 使用修改后的函数
states_padded, actions, lengths, action_nonflat = process_data(states_with_action, blocstrain, max_length=15)
pretrain_state_train, pretrain_state_valid, pretrain_len_train, pretrain_len_valid = train_test_split(states_padded, lengths, test_size=0.2, random_state=42)
pretrain_train_dataset = torch.utils.data.TensorDataset(pretrain_state_train, pretrain_len_train)
pretrain_valid_dataset = torch.utils.data.TensorDataset(pretrain_state_valid, pretrain_len_valid)
pretrain_train_loader = torch.utils.data.DataLoader(pretrain_train_dataset, batch_size=64, shuffle=True)
pretrain_valid_loader = torch.utils.data.DataLoader(pretrain_valid_dataset, batch_size=64, shuffle=False)

In [37]:
states_padded.shape

torch.Size([15218, 15, 37])

In [38]:
# the actual action length of all patients
actions.shape

torch.Size([180785, 2])

In [40]:
mean_value = torch.mean(lengths.float())

# Calculate proportions for the specified ranges
total_count = lengths.size(0)
proportion_0_5 = ((lengths >= 0) & (lengths < 5)).sum().item() / total_count
proportion_5_10 = ((lengths >= 5) & (lengths < 10)).sum().item() / total_count
proportion_10_15 = ((lengths >= 10) & (lengths < 15)).sum().item() / total_count
proportion_15_20 = ((lengths >= 15) & (lengths < 20)).sum().item() / total_count

# Print the results
print("Mean value of lengths:", mean_value.item())
print("Proportion of lengths in range 0-5:", proportion_0_5)
print("Proportion of lengths in range 5-10:", proportion_5_10)
print("Proportion of lengths in range 10-15:", proportion_10_15)
print("Proportion of lengths in range 15-20:", proportion_15_20)

Mean value of lengths: 11.879681587219238
Proportion of lengths in range 0-5: 0.04422394532790117
Proportion of lengths in range 5-10: 0.20370613746878696
Proportion of lengths in range 10-15: 0.3641740044683927
Proportion of lengths in range 15-20: 0.3878959127349192


In [71]:
lengths.shape

torch.Size([15218])

### Baseline

In [ ]:
from importlib import reload
import data_process
reload(data_process)

<module 'data_process' from '/content/drive/MyDrive/LLA/data_process.py'>

In [ ]:
ori_feature = []
for i in range(len(lengths)):
    ori_feature.append(states_padded[i, :lengths[i], :].cpu())
ori_feature = torch.cat(ori_feature, dim=0)
X_train, X_test, y_train_iol, y_test_iol, lengths_train, lengths_test = prepare_xgboost_data(ori_feature, actions[:,0],lengths)


Data shapes:
X_train shape: (144777, 37)
X_test shape: (36008, 37)
y_train shape: (144777,)
y_test shape: (36008,)
Unique action values: [0. 1. 2. 3. 4.]
lengths_train shape: torch.Size([12174])
lengths_test shape: torch.Size([3044])


In [ ]:
ori_feature.shape

torch.Size([180785, 37])

In [ ]:
xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    eval_metric='auc',
    use_label_encoder=False,
    n_estimators=300,
    learning_rate=1,
    max_depth=6,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=10,
)

In [ ]:
xgb_model = train_xgboost_classifier(xgb_model, X_train, y_train_iol, X_test, y_test_iol)
evaluate_model(xgb_model, X_train, X_test, y_train_iol, y_test_iol)

Class 0: count = 42078.0, weight = 0.6613
Class 1: count = 25614.0, weight = 1.0863
Class 2: count = 24865.0, weight = 1.1190
Class 3: count = 25297.0, weight = 1.0999
Class 4: count = 26923.0, weight = 1.0335
[0]	validation_0-auc:0.72084	validation_1-auc:0.70147


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [17:45:46] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[1]	validation_0-auc:0.74806	validation_1-auc:0.72343
[2]	validation_0-auc:0.75752	validation_1-auc:0.72978
[3]	validation_0-auc:0.76595	validation_1-auc:0.73378
[4]	validation_0-auc:0.77357	validation_1-auc:0.73499
[5]	validation_0-auc:0.78046	validation_1-auc:0.73743
[6]	validation_0-auc:0.78597	validation_1-auc:0.73743
[7]	validation_0-auc:0.79079	validation_1-auc:0.73855
[8]	validation_0-auc:0.79602	validation_1-auc:0.73903
[9]	validation_0-auc:0.80031	validation_1-auc:0.74010
[10]	validation_0-auc:0.80413	validation_1-auc:0.74031
[11]	validation_0-auc:0.80905	validation_1-auc:0.74065
[12]	validation_0-auc:0.81225	validation_1-auc:0.73997
[13]	validation_0-auc:0.81581	validation_1-auc:0.73964
[14]	validation_0-auc:0.81857	validation_1-auc:0.74003
[15]	validation_0-auc:0.82097	validation_1-auc:0.74049
[16]	validation_0-auc:0.82342	validation_1-auc:0.74073
[17]	validation_0-auc:0.82657	validation_1-auc:0.74072
[18]	validation_0-auc:0.82941	validation_1-auc:0.74063
[19]	validation_0-a

In [ ]:
import training
reload(training)

<module 'training' from '/content/drive/MyDrive/LLA/training.py'>

In [ ]:
results = evaluate_random_short_sequences(X_test, y_test_iol,lengths_test, xgb_model,n_samples = 500)

print("\nOverall Statistics:")
total_predictions = sum(len(r['predictions']) for r in results)
total_correct = sum(np.sum(r['true_labels'] == r['predictions']) for r in results)
total_confident = sum(np.sum(r['is_confident']) for r in results)

print(f"Total accuracy: {total_correct/total_predictions:.4f}")
print(f"Confident states ratio: {total_confident/total_predictions:.4f}")
print(f"Average entropy: {np.mean([np.mean(r['entropies']) for r in results]):.4f}")

流式输出内容被截断，只能显示最后 5000 行内容。
    2     |    1.0     |     4     |   0.6098   | 0.6923  | True
    3     |    4.0     |     4     |   0.9488   | 0.2276  | True
    4     |    4.0     |     4     |   0.9658   | 0.1612  | True
    5     |    1.0     |     4     |   0.8740   | 0.4496  | True

Probability distributions:
Time step | Class    0   | Class    1   | Class    2   | Class    3   | Class    4   | 
-----------------------------------------------------------------
    1     | 0.0003 | 0.0005 | 0.0002 | 0.0032 | 0.9959 | 
    2     | 0.0005 | 0.0004 | 0.0028 | 0.3865 | 0.6098 | 
    3     | 0.0005 | 0.0006 | 0.0065 | 0.0436 | 0.9488 | 
    4     | 0.0033 | 0.0000 | 0.0002 | 0.0306 | 0.9658 | 
    5     | 0.0104 | 0.0000 | 0.0103 | 0.1053 | 0.8740 | 


Sequence 252:
Time step | True Label | Predicted | Confidence | Entropy | Confident?
-----------------------------------------------------------------
    1     |    2.0     |     2     |   0.8206   | 0.6182  | True
    2     |    4.0     

### LSTM Pretraining

In [66]:
from importlib import reload
import model
reload(model)

<module 'model' from '/content/drive/MyDrive/LLA/model.py'>

In [67]:
pretrain_model = model.LSTM_pretrain(
    input_dim=37,
    hidden_dim=512
).to(device)
initialize_weights(pretrain_model)

loss_fn = nn.MSELoss()
optimizer = torch.optim.AdamW(pretrain_model.parameters(), lr=1e-3)

# early stop
patience = 5
min_valid_loss = float('inf')
patience_counter = 0
best_model_state = None

for epoch in range(100):
    pretrain_model.eval()
    valid_loss = 0
    with torch.no_grad():
        for i, (batch, leng) in enumerate(pretrain_valid_loader):
            batch = batch.to(device)
            leng = leng.to(device)
            pred = pretrain_model(batch, leng)
            loss = loss_fn(pred, batch)
            valid_loss += loss.item()
        valid_loss = valid_loss / len(pretrain_valid_loader)
        print(f"Valid Epoch {epoch}, Loss: {valid_loss}")

    pretrain_model.train()
    total_loss = 0
    for i, (batch, leng) in enumerate(pretrain_train_loader):
        batch = batch.to(device)
        leng = leng.to(device)
        optimizer.zero_grad()
        pred = pretrain_model(batch, leng)
        loss = loss_fn(pred, batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Train Epoch {epoch}, Loss: {total_loss / len(pretrain_train_loader)}")

    if valid_loss < min_valid_loss:
        min_valid_loss = valid_loss
        best_model_state = pretrain_model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"Early stopping triggered after epoch {epoch}")
        pretrain_model.load_state_dict(best_model_state)
        break

Valid Epoch 0, Loss: 0.853223287810882
Train Epoch 0, Loss: 0.14635731012155717
Valid Epoch 1, Loss: 0.08277115235493208
Train Epoch 1, Loss: 0.02868905483808427
Valid Epoch 2, Loss: 0.07045063638361171
Train Epoch 2, Loss: 0.022721129112403975
Valid Epoch 3, Loss: 0.06699704397760797
Train Epoch 3, Loss: 0.019061616127603535
Valid Epoch 4, Loss: 0.0632873745150088
Train Epoch 4, Loss: 0.01662911503412395
Valid Epoch 5, Loss: 0.06107181507832138
Train Epoch 5, Loss: 0.01485472297172808
Valid Epoch 6, Loss: 0.05921865169996939
Train Epoch 6, Loss: 0.01402260591544172
Valid Epoch 7, Loss: 0.05946914024389116
Train Epoch 7, Loss: 0.013496366758582167
Valid Epoch 8, Loss: 0.059005605722632026
Train Epoch 8, Loss: 0.011602120474590407
Valid Epoch 9, Loss: 0.06169938362048318
Train Epoch 9, Loss: 0.011277086759737025
Valid Epoch 10, Loss: 0.05869364562871245
Train Epoch 10, Loss: 0.009721222373227837
Valid Epoch 11, Loss: 0.057845751113442624
Train Epoch 11, Loss: 0.010123364486870555
Valid 

In [68]:
output = pretrain_model.forward_embedding(states_padded.to(device), lengths)
X_train, X_test, y_train_iol, y_test_iol, lengths_train, lengths_test = prepare_xgboost_data(output, actions[:,0],lengths)


Data shapes:
X_train shape: (144777, 512)
X_test shape: (36008, 512)
y_train shape: (144777,)
y_test shape: (36008,)
Unique action values: [0. 1. 2. 3. 4.]
lengths_train shape: torch.Size([12174])
lengths_test shape: torch.Size([3044])


In [69]:
xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    eval_metric='auc',
    use_label_encoder=False,
    n_estimators=300,
    learning_rate=1,
    max_depth=6,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=10,
)
xgb_model = train_xgboost_classifier(xgb_model, X_train, y_train_iol, X_test, y_test_iol)
evaluate_model(xgb_model, X_train, X_test, y_train_iol, y_test_iol)

Class 0: count = 42078.0, weight = 0.6613
Class 1: count = 25614.0, weight = 1.0863
Class 2: count = 24865.0, weight = 1.1190
Class 3: count = 25297.0, weight = 1.0999
Class 4: count = 26923.0, weight = 1.0335


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:41:25] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-auc:0.69093	validation_1-auc:0.66309
[1]	validation_0-auc:0.72067	validation_1-auc:0.68275
[2]	validation_0-auc:0.73794	validation_1-auc:0.69079
[3]	validation_0-auc:0.75108	validation_1-auc:0.69485
[4]	validation_0-auc:0.76212	validation_1-auc:0.69901
[5]	validation_0-auc:0.76985	validation_1-auc:0.70052
[6]	validation_0-auc:0.77826	validation_1-auc:0.70287
[7]	validation_0-auc:0.78628	validation_1-auc:0.70473
[8]	validation_0-auc:0.79227	validation_1-auc:0.70538
[9]	validation_0-auc:0.79854	validation_1-auc:0.70585
[10]	validation_0-auc:0.80447	validation_1-auc:0.70720
[11]	validation_0-auc:0.80964	validation_1-auc:0.70834
[12]	validation_0-auc:0.81506	validation_1-auc:0.70933
[13]	validation_0-auc:0.82023	validation_1-auc:0.70939
[14]	validation_0-auc:0.82495	validation_1-auc:0.71015
[15]	validation_0-auc:0.82862	validation_1-auc:0.71093
[16]	validation_0-auc:0.83271	validation_1-auc:0.71137
[17]	validation_0-auc:0.83681	validation_1-auc:0.71193
[18]	validation_0-au

In [70]:
results = evaluate_random_short_sequences(X_test, y_test_iol,lengths_test, xgb_model,n_samples = 500)

print("\nOverall Statistics:")
total_predictions = sum(len(r['predictions']) for r in results)
total_correct = sum(np.sum(r['true_labels'] == r['predictions']) for r in results)
total_confident = sum(np.sum(r['is_confident']) for r in results)

print(f"Total accuracy: {total_correct/total_predictions:.4f}")
print(f"Confident states ratio: {total_confident/total_predictions:.4f}")
print(f"Average entropy: {np.mean([np.mean(r['entropies']) for r in results]):.4f}")

流式输出内容被截断，只能显示最后 5000 行内容。
    2     |    0.0     |     1     |   0.7305   | 0.9073  | False
    3     |    0.0     |     2     |   0.3235   | 1.5121  | False
    4     |    0.0     |     1     |   0.5546   | 1.2359  | False
    5     |    0.0     |     0     |   0.9970   | 0.0213  | True

Probability distributions:
Time step | Class    0   | Class    1   | Class    2   | Class    3   | Class    4   | 
-----------------------------------------------------------------
    1     | 0.1006 | 0.7317 | 0.0827 | 0.0320 | 0.0531 | 
    2     | 0.0204 | 0.7305 | 0.0496 | 0.1317 | 0.0677 | 
    3     | 0.1104 | 0.1873 | 0.3235 | 0.2752 | 0.1037 | 
    4     | 0.1647 | 0.5546 | 0.0671 | 0.1751 | 0.0386 | 
    5     | 0.9970 | 0.0028 | 0.0002 | 0.0000 | 0.0000 | 


Sequence 252:
Time step | True Label | Predicted | Confidence | Entropy | Confident?
-----------------------------------------------------------------
    1     |    4.0     |     4     |   0.2606   | 1.5877  | False
    2     |    4.0 

### LSTM WITHOUT Pretraining

In [72]:
pretrain_model = model.LSTM_pretrain(
    input_dim=37,
    hidden_dim=512,
).to(device)
initialize_weights(pretrain_model)

In [73]:
output = pretrain_model.forward_embedding(states_padded.to(device), lengths)
X_train, X_test, y_train_iol, y_test_iol, lengths_train, lengths_test= prepare_xgboost_data(output, actions[:,0],lengths )


Data shapes:
X_train shape: (144777, 512)
X_test shape: (36008, 512)
y_train shape: (144777,)
y_test shape: (36008,)
Unique action values: [0. 1. 2. 3. 4.]
lengths_train shape: torch.Size([12174])
lengths_test shape: torch.Size([3044])


In [74]:
xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    eval_metric='auc',
    use_label_encoder=False,
    n_estimators=300,
    learning_rate=1,
    max_depth=6,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=10,
)

In [75]:
xgb_model = train_xgboost_classifier(xgb_model, X_train, y_train_iol, X_test, y_test_iol)
evaluate_model(xgb_model, X_train, X_test, y_train_iol, y_test_iol)

Class 0: count = 42078.0, weight = 0.6613
Class 1: count = 25614.0, weight = 1.0863
Class 2: count = 24865.0, weight = 1.1190
Class 3: count = 25297.0, weight = 1.0999
Class 4: count = 26923.0, weight = 1.0335


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [18:39:20] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-auc:0.69334	validation_1-auc:0.66067
[1]	validation_0-auc:0.71888	validation_1-auc:0.67325
[2]	validation_0-auc:0.73512	validation_1-auc:0.68002
[3]	validation_0-auc:0.74676	validation_1-auc:0.68496
[4]	validation_0-auc:0.75868	validation_1-auc:0.68763
[5]	validation_0-auc:0.76893	validation_1-auc:0.68929
[6]	validation_0-auc:0.77735	validation_1-auc:0.69053
[7]	validation_0-auc:0.78511	validation_1-auc:0.69158
[8]	validation_0-auc:0.79232	validation_1-auc:0.69286
[9]	validation_0-auc:0.79877	validation_1-auc:0.69366
[10]	validation_0-auc:0.80493	validation_1-auc:0.69430
[11]	validation_0-auc:0.80988	validation_1-auc:0.69430
[12]	validation_0-auc:0.81482	validation_1-auc:0.69531
[13]	validation_0-auc:0.82007	validation_1-auc:0.69601
[14]	validation_0-auc:0.82458	validation_1-auc:0.69624
[15]	validation_0-auc:0.82896	validation_1-auc:0.69630
[16]	validation_0-auc:0.83328	validation_1-auc:0.69679
[17]	validation_0-auc:0.83743	validation_1-auc:0.69694
[18]	validation_0-au

In [76]:
results = evaluate_random_short_sequences(X_test, y_test_iol,lengths_test, xgb_model,n_samples = 500)

print("\nOverall Statistics:")
total_predictions = sum(len(r['predictions']) for r in results)
total_correct = sum(np.sum(r['true_labels'] == r['predictions']) for r in results)
total_confident = sum(np.sum(r['is_confident']) for r in results)

print(f"Total accuracy: {total_correct/total_predictions:.4f}")
print(f"Confident states ratio: {total_confident/total_predictions:.4f}")
print(f"Average entropy: {np.mean([np.mean(r['entropies']) for r in results]):.4f}")

流式输出内容被截断，只能显示最后 5000 行内容。
    2     |    3.0     |     2     |   0.6207   | 1.1000  | False
    3     |    3.0     |     3     |   0.6289   | 1.1229  | False
    4     |    3.0     |     1     |   0.5985   | 1.1742  | False
    5     |    4.0     |     3     |   0.3989   | 1.2332  | False

Probability distributions:
Time step | Class    0   | Class    1   | Class    2   | Class    3   | Class    4   | 
-----------------------------------------------------------------
    1     | 0.5267 | 0.0224 | 0.0658 | 0.3502 | 0.0349 | 
    2     | 0.0848 | 0.1083 | 0.6207 | 0.1754 | 0.0107 | 
    3     | 0.0238 | 0.1168 | 0.0879 | 0.6289 | 0.1425 | 
    4     | 0.0570 | 0.5985 | 0.1934 | 0.0577 | 0.0934 | 
    5     | 0.0175 | 0.3460 | 0.2090 | 0.3989 | 0.0286 | 


Sequence 252:
Time step | True Label | Predicted | Confidence | Entropy | Confident?
-----------------------------------------------------------------
    1     |    3.0     |     2     |   0.8606   | 0.5521  | True
    2     |    3.0 

### LSTM_Attention Pretraining

In [25]:
from importlib import reload
import model
reload(model)

<module 'model' from '/content/drive/MyDrive/LLA/model.py'>

In [41]:
pretrain_model = model.LSTMAttentionModel(
    input_dim=37,
    hidden_dim=512,
    attn_layers=1,
).to(device)
initialize_weights(pretrain_model)
print(pretrain_model)

loss_fn = nn.MSELoss()
optimizer = torch.optim.AdamW(pretrain_model.parameters(), lr=1e-3)

# early stop
patience = 5
min_valid_loss = float('inf')
patience_counter = 0
best_model_state = None

for epoch in range(100):
    pretrain_model.eval()
    valid_loss = 0
    with torch.no_grad():
        for i, (batch, leng) in enumerate(pretrain_valid_loader):
            batch = batch.to(device)
            leng = leng.to(device)
            pred = pretrain_model(batch, leng)
            loss = loss_fn(pred, batch)
            valid_loss += loss.item()
        valid_loss = valid_loss / len(pretrain_valid_loader)
        print(f"Valid Epoch {epoch}, Loss: {valid_loss}")

    pretrain_model.train()
    total_loss = 0
    for i, (batch, leng) in enumerate(pretrain_train_loader):
        batch = batch.to(device)
        leng = leng.to(device)
        optimizer.zero_grad()
        pred = pretrain_model(batch, leng)
        loss = loss_fn(pred, batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Train Epoch {epoch}, Loss: {total_loss / len(pretrain_train_loader)}")

    if valid_loss < min_valid_loss:
        min_valid_loss = valid_loss
        best_model_state = pretrain_model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"Early stopping triggered after epoch {epoch}")
        pretrain_model.load_state_dict(best_model_state)
        break

LSTMAttentionModel(
  (lstm): LSTM(37, 512, batch_first=True)
  (atten_layers): Sequential(
    (0): CausalAttention(
      (query_fc): Linear(in_features=512, out_features=512, bias=True)
      (key_fc): Linear(in_features=512, out_features=512, bias=True)
      (value_fc): Linear(in_features=512, out_features=512, bias=True)
      (softmax): Softmax(dim=-1)
    )
  )
  (prediction_head): Linear(in_features=512, out_features=37, bias=True)
)
Valid Epoch 0, Loss: 0.8471807080010573
Train Epoch 0, Loss: 0.10133236122478521
Valid Epoch 1, Loss: 0.059901178499179274
Train Epoch 1, Loss: 0.03520620000580355
Valid Epoch 2, Loss: 0.07379145716549829
Train Epoch 2, Loss: 0.04401164532746359
Valid Epoch 3, Loss: 0.08217277521422754
Train Epoch 3, Loss: 0.021976412698158417
Valid Epoch 4, Loss: 0.058186957418608166
Train Epoch 4, Loss: 0.014072947790456146
Valid Epoch 5, Loss: 0.055776590471699215
Train Epoch 5, Loss: 0.011411320626569903
Valid Epoch 6, Loss: 0.05590987248676053
Train Epoch 6, 

In [42]:
pretrain_model.eval()
output = pretrain_model.forward_embedding(states_padded.to(device), lengths)
X_train, X_test, y_train_iol, y_test_iol,lengths_train, lengths_test = prepare_xgboost_data(output, actions[:,0], lengths)


Data shapes:
X_train shape: (144777, 512)
X_test shape: (36008, 512)
y_train shape: (144777,)
y_test shape: (36008,)
Unique action values: [0. 1. 2. 3. 4.]
lengths_train shape: torch.Size([12174])
lengths_test shape: torch.Size([3044])


In [46]:
xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    eval_metric='auc',
    use_label_encoder=False,
    n_estimators=300,
    learning_rate=1,
    max_depth=6,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=10,
)

In [47]:
xgb_model = train_xgboost_classifier(xgb_model, X_train, y_train_iol, X_test, y_test_iol)
evaluate_model(xgb_model, X_train, X_test, y_train_iol, y_test_iol)

Class 0: count = 42078.0, weight = 0.6613
Class 1: count = 25614.0, weight = 1.0863
Class 2: count = 24865.0, weight = 1.1190
Class 3: count = 25297.0, weight = 1.0999
Class 4: count = 26923.0, weight = 1.0335


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:51:21] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-auc:0.69699	validation_1-auc:0.67398
[1]	validation_0-auc:0.72269	validation_1-auc:0.68814
[2]	validation_0-auc:0.73809	validation_1-auc:0.69508
[3]	validation_0-auc:0.75036	validation_1-auc:0.69848
[4]	validation_0-auc:0.76019	validation_1-auc:0.70124
[5]	validation_0-auc:0.76854	validation_1-auc:0.70351
[6]	validation_0-auc:0.77594	validation_1-auc:0.70436
[7]	validation_0-auc:0.78156	validation_1-auc:0.70516
[8]	validation_0-auc:0.78782	validation_1-auc:0.70626
[9]	validation_0-auc:0.79417	validation_1-auc:0.70751
[10]	validation_0-auc:0.79907	validation_1-auc:0.70895
[11]	validation_0-auc:0.80488	validation_1-auc:0.70893
[12]	validation_0-auc:0.80990	validation_1-auc:0.70952
[13]	validation_0-auc:0.81485	validation_1-auc:0.70907
[14]	validation_0-auc:0.81902	validation_1-auc:0.70985
[15]	validation_0-auc:0.82279	validation_1-auc:0.70999
[16]	validation_0-auc:0.82665	validation_1-auc:0.71002
[17]	validation_0-auc:0.83117	validation_1-auc:0.70989
[18]	validation_0-au

In [48]:
results = evaluate_random_short_sequences(X_test, y_test_iol,lengths_test, xgb_model,n_samples = 500)

print("\nOverall Statistics:")
total_predictions = sum(len(r['predictions']) for r in results)
total_correct = sum(np.sum(r['true_labels'] == r['predictions']) for r in results)
total_confident = sum(np.sum(r['is_confident']) for r in results)

print(f"Total accuracy: {total_correct/total_predictions:.4f}")
print(f"Confident states ratio: {total_confident/total_predictions:.4f}")
print(f"Average entropy: {np.mean([np.mean(r['entropies']) for r in results]):.4f}")

流式输出内容被截断，只能显示最后 5000 行内容。
    2     |    2.0     |     3     |   0.8151   | 0.6657  | True
    3     |    2.0     |     4     |   0.4177   | 1.2533  | False
    4     |    4.0     |     2     |   0.3266   | 1.3852  | False
    5     |    4.0     |     4     |   0.5298   | 1.2160  | False

Probability distributions:
Time step | Class    0   | Class    1   | Class    2   | Class    3   | Class    4   | 
-----------------------------------------------------------------
    1     | 0.4392 | 0.3654 | 0.1389 | 0.0556 | 0.0009 | 
    2     | 0.0099 | 0.0228 | 0.0339 | 0.8151 | 0.1184 | 
    3     | 0.3624 | 0.0109 | 0.0976 | 0.1115 | 0.4177 | 
    4     | 0.2477 | 0.0066 | 0.3266 | 0.2633 | 0.1558 | 
    5     | 0.1090 | 0.0088 | 0.1241 | 0.2282 | 0.5298 | 


Sequence 252:
Time step | True Label | Predicted | Confidence | Entropy | Confident?
-----------------------------------------------------------------
    1     |    1.0     |     0     |   0.4036   | 1.3918  | False
    2     |    1.0 

## **LSTM+ attention without pretaining**

In [56]:
pretrain_model = model.LSTMAttentionModel(
    input_dim=37,
    hidden_dim=512,
).to(device)
initialize_weights(pretrain_model)

In [57]:
output = pretrain_model.forward_embedding(states_padded.to(device), lengths)
X_train, X_test, y_train_iol, y_test_iol, lengths_train, lengths_test= prepare_xgboost_data(output, actions[:,0],lengths )


Data shapes:
X_train shape: (144777, 512)
X_test shape: (36008, 512)
y_train shape: (144777,)
y_test shape: (36008,)
Unique action values: [0. 1. 2. 3. 4.]
lengths_train shape: torch.Size([12174])
lengths_test shape: torch.Size([3044])


In [58]:
xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    eval_metric='auc',
    use_label_encoder=False,
    n_estimators=300,
    learning_rate=1,
    max_depth=6,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=10,
)

In [59]:
xgb_model = train_xgboost_classifier(xgb_model, X_train, y_train_iol, X_test, y_test_iol)
evaluate_model(xgb_model, X_train, X_test, y_train_iol, y_test_iol)

Class 0: count = 42078.0, weight = 0.6613
Class 1: count = 25614.0, weight = 1.0863
Class 2: count = 24865.0, weight = 1.1190
Class 3: count = 25297.0, weight = 1.0999
Class 4: count = 26923.0, weight = 1.0335


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:44:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-auc:0.67935	validation_1-auc:0.63076
[1]	validation_0-auc:0.70759	validation_1-auc:0.64285
[2]	validation_0-auc:0.72527	validation_1-auc:0.64687
[3]	validation_0-auc:0.74143	validation_1-auc:0.64913
[4]	validation_0-auc:0.75323	validation_1-auc:0.65096
[5]	validation_0-auc:0.76506	validation_1-auc:0.65214
[6]	validation_0-auc:0.77467	validation_1-auc:0.65340
[7]	validation_0-auc:0.78386	validation_1-auc:0.65450
[8]	validation_0-auc:0.79211	validation_1-auc:0.65633
[9]	validation_0-auc:0.79972	validation_1-auc:0.65676
[10]	validation_0-auc:0.80691	validation_1-auc:0.65752
[11]	validation_0-auc:0.81359	validation_1-auc:0.65662
[12]	validation_0-auc:0.82041	validation_1-auc:0.65649
[13]	validation_0-auc:0.82619	validation_1-auc:0.65583
[14]	validation_0-auc:0.83201	validation_1-auc:0.65643
[15]	validation_0-auc:0.83663	validation_1-auc:0.65679
[16]	validation_0-auc:0.84211	validation_1-auc:0.65646
[17]	validation_0-auc:0.84646	validation_1-auc:0.65670
[18]	validation_0-au

In [60]:
results = evaluate_random_short_sequences(X_test, y_test_iol,lengths_test, xgb_model,n_samples = 500)

print("\nOverall Statistics:")
total_predictions = sum(len(r['predictions']) for r in results)
total_correct = sum(np.sum(r['true_labels'] == r['predictions']) for r in results)
total_confident = sum(np.sum(r['is_confident']) for r in results)

print(f"Total accuracy: {total_correct/total_predictions:.4f}")
print(f"Confident states ratio: {total_confident/total_predictions:.4f}")
print(f"Average entropy: {np.mean([np.mean(r['entropies']) for r in results]):.4f}")

流式输出内容被截断，只能显示最后 5000 行内容。
    2     |    1.0     |     0     |   0.9998   | 0.0020  | True
    3     |    1.0     |     0     |   0.9685   | 0.1412  | True
    4     |    0.0     |     0     |   0.9802   | 0.1017  | True
    5     |    2.0     |     0     |   0.9178   | 0.3135  | True

Probability distributions:
Time step | Class    0   | Class    1   | Class    2   | Class    3   | Class    4   | 
-----------------------------------------------------------------
    1     | 1.0000 | 0.0000 | 0.0000 | 0.0000 | 0.0000 | 
    2     | 0.9998 | 0.0000 | 0.0000 | 0.0000 | 0.0002 | 
    3     | 0.9685 | 0.0001 | 0.0000 | 0.0001 | 0.0313 | 
    4     | 0.9802 | 0.0000 | 0.0002 | 0.0008 | 0.0187 | 
    5     | 0.9178 | 0.0012 | 0.0007 | 0.0052 | 0.0752 | 


Sequence 252:
Time step | True Label | Predicted | Confidence | Entropy | Confident?
-----------------------------------------------------------------
    1     |    0.0     |     0     |   0.4625   | 1.0881  | False
    2     |    0.0    

## **GPT pretraining**

---

In [ ]:
from importlib import reload
import model
reload(model)

<module 'model' from '/rds/general/user/yz9023/home/LLA/model.py'>

In [61]:
input_dim = 37
hidden_dim = 512
num_layers = 1
num_heads = 1
dropout = 0.05

pretrain_model = model.GPTCausalAttentionModel(input_dim=input_dim, hidden_dim=hidden_dim,
                                num_layers=num_layers, num_heads=num_heads, dropout=dropout).to(device)
initialize_weights(pretrain_model)
loss_fn = nn.MSELoss()
optimizer = torch.optim.AdamW(pretrain_model.parameters(), lr=1e-3)

# early stop
patience = 10
min_valid_loss = float('inf')
patience_counter = 0
best_model_state = None

for epoch in range(100):
    pretrain_model.eval()
    valid_loss = 0
    with torch.no_grad():
        for i, (batch, leng) in enumerate(pretrain_valid_loader):
            batch = batch.to(device)
            pred = pretrain_model(batch)
            loss = loss_fn(pred, batch)
            valid_loss += loss.item()
        valid_loss = valid_loss / len(pretrain_valid_loader)
        print(f"Valid Epoch {epoch}, Loss: {valid_loss}")

    pretrain_model.train()
    total_loss = 0
    for i, (batch, leng) in enumerate(pretrain_train_loader):
        batch = batch.to(device)
        optimizer.zero_grad()
        pred = pretrain_model(batch)
        loss = loss_fn(pred, batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Train Epoch {epoch}, Loss: {total_loss / len(pretrain_train_loader)}")

    if valid_loss < min_valid_loss:
        min_valid_loss = valid_loss
        best_model_state = pretrain_model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"Early stopping triggered after epoch {epoch}")
        pretrain_model.load_state_dict(best_model_state)
        break

Valid Epoch 0, Loss: 2.716746985912323
Train Epoch 0, Loss: 0.21002874835467464
Valid Epoch 1, Loss: 0.10733650263864547
Train Epoch 1, Loss: 0.06908723448661609
Valid Epoch 2, Loss: 0.08956250157401276
Train Epoch 2, Loss: 0.05137578442361186
Valid Epoch 3, Loss: 0.10209922793243702
Train Epoch 3, Loss: 0.05089180114927716
Valid Epoch 4, Loss: 0.091360782054835
Train Epoch 4, Loss: 0.04419407388682999
Valid Epoch 5, Loss: 0.11865724363209058
Train Epoch 5, Loss: 0.053937806167605656
Valid Epoch 6, Loss: 0.09017746018556257
Train Epoch 6, Loss: 0.044190391942039525
Valid Epoch 7, Loss: 0.09726525841203208
Train Epoch 7, Loss: 0.07970889748917669
Valid Epoch 8, Loss: 0.5563816825548807
Train Epoch 8, Loss: 0.09928809867498918
Valid Epoch 9, Loss: 0.09283847758585277
Train Epoch 9, Loss: 0.04401027468028259
Valid Epoch 10, Loss: 0.09267905437930797
Train Epoch 10, Loss: 0.05213432573025841
Valid Epoch 11, Loss: 0.09565807227045298
Train Epoch 11, Loss: 0.041887937481783294
Valid Epoch 12

In [63]:
pretrain_model.eval()
output = pretrain_model.forward_embedding(states_padded.to(device), lengths)
X_train, X_test, y_train_iol, y_test_iol, lengths_train, lengths_test= prepare_xgboost_data(output, actions[:,0],lengths )


Data shapes:
X_train shape: (144777, 512)
X_test shape: (36008, 512)
y_train shape: (144777,)
y_test shape: (36008,)
Unique action values: [0. 1. 2. 3. 4.]
lengths_train shape: torch.Size([12174])
lengths_test shape: torch.Size([3044])


In [64]:
xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    eval_metric='auc',
    use_label_encoder=False,
    n_estimators=300,
    learning_rate=1,
    max_depth=8,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=10,
)
xgb_model = train_xgboost_classifier(xgb_model, X_train, y_train_iol, X_test, y_test_iol)
evaluate_model(xgb_model, X_train, X_test, y_train_iol, y_test_iol)

Class 0: count = 42078.0, weight = 0.6613
Class 1: count = 25614.0, weight = 1.0863
Class 2: count = 24865.0, weight = 1.1190
Class 3: count = 25297.0, weight = 1.0999
Class 4: count = 26923.0, weight = 1.0335


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:04:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-auc:0.72265	validation_1-auc:0.66539
[1]	validation_0-auc:0.76216	validation_1-auc:0.67975
[2]	validation_0-auc:0.78433	validation_1-auc:0.68445
[3]	validation_0-auc:0.80311	validation_1-auc:0.68984
[4]	validation_0-auc:0.81701	validation_1-auc:0.69209
[5]	validation_0-auc:0.83069	validation_1-auc:0.69342
[6]	validation_0-auc:0.84194	validation_1-auc:0.69438
[7]	validation_0-auc:0.85198	validation_1-auc:0.69480
[8]	validation_0-auc:0.86078	validation_1-auc:0.69611
[9]	validation_0-auc:0.86819	validation_1-auc:0.69704
[10]	validation_0-auc:0.87635	validation_1-auc:0.69752
[11]	validation_0-auc:0.88337	validation_1-auc:0.69742
[12]	validation_0-auc:0.89108	validation_1-auc:0.69738
[13]	validation_0-auc:0.89719	validation_1-auc:0.69778
[14]	validation_0-auc:0.90281	validation_1-auc:0.69696
[15]	validation_0-auc:0.90894	validation_1-auc:0.69692
[16]	validation_0-auc:0.91382	validation_1-auc:0.69725
[17]	validation_0-auc:0.91831	validation_1-auc:0.69741
[18]	validation_0-au

---

In [65]:
results = evaluate_random_short_sequences(X_test, y_test_iol,lengths_test, xgb_model,n_samples = 500)

print("\nOverall Statistics:")
total_predictions = sum(len(r['predictions']) for r in results)
total_correct = sum(np.sum(r['true_labels'] == r['predictions']) for r in results)
total_confident = sum(np.sum(r['is_confident']) for r in results)

print(f"Total accuracy: {total_correct/total_predictions:.4f}")
print(f"Confident states ratio: {total_confident/total_predictions:.4f}")
print(f"Average entropy: {np.mean([np.mean(r['entropies']) for r in results]):.4f}")

流式输出内容被截断，只能显示最后 5000 行内容。
    2     |    0.0     |     1     |   0.8535   | 0.4541  | True
    3     |    0.0     |     1     |   0.5612   | 0.9666  | False
    4     |    0.0     |     1     |   0.9939   | 0.0395  | True
    5     |    0.0     |     0     |   0.5223   | 0.7929  | False

Probability distributions:
Time step | Class    0   | Class    1   | Class    2   | Class    3   | Class    4   | 
-----------------------------------------------------------------
    1     | 0.0547 | 0.8825 | 0.0627 | 0.0000 | 0.0000 | 
    2     | 0.1380 | 0.8535 | 0.0069 | 0.0009 | 0.0007 | 
    3     | 0.1315 | 0.5612 | 0.3053 | 0.0004 | 0.0016 | 
    4     | 0.0008 | 0.9939 | 0.0053 | 0.0000 | 0.0000 | 
    5     | 0.5223 | 0.4531 | 0.0237 | 0.0008 | 0.0001 | 


Sequence 252:
Time step | True Label | Predicted | Confidence | Entropy | Confident?
-----------------------------------------------------------------
    1     |    4.0     |     1     |   0.7303   | 0.9086  | False
    2     |    4.0  

## **LSTM**

In [ ]:
def reshape_for_xgboost(data):
    samples = data.shape[0]
    return data.reshape(samples, -1)

In [ ]:
lstm_ouput.shape

torch.Size([15218, 15, 128])

In [ ]:
action_nonflat[:,0].shape

torch.Size([15218, 2])

In [ ]:
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = prepare_xgboost_data(lstm_ouput, action_nonflat[:,0])
X_train_reshaped = reshape_for_xgboost(X_train_lstm)
X_test_reshaped = reshape_for_xgboost(X_test_lstm)


Data shapes:
X_train shape: (12174, 15, 128)
X_test shape: (3044, 15, 128)
y_train shape: (12174, 2)
y_test shape: (3044, 2)
Unique action values: [0. 1. 2. 3. 4.]


In [ ]:
def train_xgboost_classifier(X_train, y_train, X_test, y_test):
    """
    训练XGBoost分类器并评估
    """
    y_train = y_train.astype(int)
    y_test = y_test.astype(int)
    sample_weights = calculate_class_weights(y_train)

    xgb_model = xgb.XGBClassifier(
        objective='multi:softprob',
        eval_metric='auc',
        use_label_encoder=False,
        n_estimators=200,
        learning_rate=0.1,
        max_depth=6,
        min_child_weight=5,
        subsample=0.8,
        colsample_bytree=0.8,
        n_jobs=8,
    )

    xgb_model.fit(
        X_train,
        y_train,
        sample_weight=sample_weights,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=True
    )

    return xgb_model
xgb_model_lstm = train_xgboost_classifier(X_train_reshaped, y_train_lstm[:,0], X_test_reshaped, y_test_lstm[:,0])
evaluate_model(xgb_model_lstm, X_train_reshaped, X_test_reshaped, y_train_lstm[:,0], y_test_lstm[:,0])

Class 0: count = 7543.0, weight = 0.1847
Class 1: count = 1116.0, weight = 1.2481
Class 2: count = 1172.0, weight = 1.1884
Class 3: count = 1148.0, weight = 1.2133
Class 4: count = 1195.0, weight = 1.1656
[0]	validation_0-auc:0.86273	validation_1-auc:0.78186
[1]	validation_0-auc:0.90009	validation_1-auc:0.81211
[2]	validation_0-auc:0.91567	validation_1-auc:0.82536
[3]	validation_0-auc:0.92506	validation_1-auc:0.83526
[4]	validation_0-auc:0.93133	validation_1-auc:0.83978
[5]	validation_0-auc:0.93770	validation_1-auc:0.84319
[6]	validation_0-auc:0.94197	validation_1-auc:0.84766
[7]	validation_0-auc:0.94432	validation_1-auc:0.85078
[8]	validation_0-auc:0.94709	validation_1-auc:0.85222
[9]	validation_0-auc:0.94970	validation_1-auc:0.85472
[10]	validation_0-auc:0.95236	validation_1-auc:0.85700
[11]	validation_0-auc:0.95393	validation_1-auc:0.85829
[12]	validation_0-auc:0.95588	validation_1-auc:0.86048
[13]	validation_0-auc:0.95741	validation_1-auc:0.86158
[14]	validation_0-auc:0.95889	valid

In [ ]:
lstm_ouput.shape

torch.Size([15218, 15, 128])

In [ ]:
actions[:,0].shape

torch.Size([180785])

In [ ]:
all_lstm_outputs = []
for i, lstm_o in enumerate(lstm_ouput):
    # 只取实际长度的action数据
    actual_length = lengths[i]
    all_lstm_outputs.append(lstm_o[:actual_length])
all_lstm_outputs = torch.cat(all_lstm_outputs, dim=0)

In [ ]:
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = prepare_xgboost_data(all_lstm_outputs, actions[:,0])
X_train_reshaped = reshape_for_xgboost(X_train_lstm)
X_test_reshaped = reshape_for_xgboost(X_test_lstm)

def train_xgboost_classifier(X_train, y_train, X_test, y_test):
    """
    训练XGBoost分类器并评估
    """
    y_train = y_train.astype(int)
    y_test = y_test.astype(int)
    sample_weights = calculate_class_weights(y_train)

    xgb_model = xgb.XGBClassifier(
        objective='multi:softprob',
        eval_metric='auc',
        use_label_encoder=False,
        n_estimators=200,
        learning_rate=0.1,
        max_depth=6,
        min_child_weight=5,
        subsample=0.8,
        colsample_bytree=0.8,
        n_jobs=8,
    )

    xgb_model.fit(
        X_train,
        y_train,
        sample_weight=sample_weights,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=True
    )

    return xgb_model
xgb_model_lstm = train_xgboost_classifier(X_train_reshaped, y_train_lstm, X_test_reshaped, y_test_lstm)
evaluate_model(xgb_model_lstm, X_train_reshaped, X_test_reshaped, y_train_lstm, y_test_lstm)


Data shapes:
X_train shape: (144628, 128)
X_test shape: (36157, 128)
y_train shape: (144628,)
y_test shape: (36157,)
Unique action values: [0. 1. 2. 3. 4.]
Class 0: count = 41625.0, weight = 0.6692
Class 1: count = 25552.0, weight = 1.0902
Class 2: count = 24984.0, weight = 1.1149
Class 3: count = 25427.0, weight = 1.0955
Class 4: count = 27040.0, weight = 1.0302
[0]	validation_0-auc:0.68014	validation_1-auc:0.65880
[1]	validation_0-auc:0.69865	validation_1-auc:0.67516
[2]	validation_0-auc:0.70953	validation_1-auc:0.68352
[3]	validation_0-auc:0.71655	validation_1-auc:0.68868
[4]	validation_0-auc:0.72006	validation_1-auc:0.69199
[5]	validation_0-auc:0.72342	validation_1-auc:0.69389
[6]	validation_0-auc:0.72556	validation_1-auc:0.69583
[7]	validation_0-auc:0.72772	validation_1-auc:0.69728
[8]	validation_0-auc:0.72966	validation_1-auc:0.69861
[9]	validation_0-auc:0.73177	validation_1-auc:0.69970
[10]	validation_0-auc:0.73373	validation_1-auc:0.70080
[11]	validation_0-auc:0.73550	validati

In [ ]:
results = evaluate_random_short_sequences(X_test_reshaped, y_test_lstm[:,0], xgb_model_lstm, n_samples = 500)

print("\nOverall Statistics:")
total_predictions = sum(len(r['predictions']) for r in results)
total_correct = sum(np.sum(r['true_labels'] == r['predictions']) for r in results)
total_confident = sum(np.sum(r['is_confident']) for r in results)

print(f"Total accuracy: {total_correct/total_predictions:.4f}")
print(f"Confident states ratio: {total_confident/total_predictions:.4f}")
print(f"Average entropy: {np.mean([np.mean(r['entropies']) for r in results]):.4f}")

流式输出内容被截断，只能显示最后 5000 行内容。
    2     |    1.0     |     1     |   0.8267   | 0.6394  | True
    3     |    0.0     |     0     |   0.9886   | 0.0714  | True
    4     |    2.0     |     0     |   0.8899   | 0.4754  | True
    5     |    2.0     |     3     |   0.3686   | 1.4469  | False

Probability distributions:
Time step | Class    0   | Class    1   | Class    2   | Class    3   | Class    4   | 
-----------------------------------------------------------------
    1     | 0.9940 | 0.0028 | 0.0003 | 0.0014 | 0.0015 | 
    2     | 0.0872 | 0.8267 | 0.0633 | 0.0200 | 0.0027 | 
    3     | 0.9886 | 0.0013 | 0.0005 | 0.0009 | 0.0087 | 
    4     | 0.8899 | 0.0582 | 0.0279 | 0.0110 | 0.0131 | 
    5     | 0.0966 | 0.0614 | 0.2282 | 0.3686 | 0.2452 | 


Sequence 252 (starting from index 890):
Time step | True Label | Predicted | Confidence | Entropy | Confident?
-----------------------------------------------------------------
    1     |    0.0     |     0     |   0.9736   | 0.1262  | T